# Introduction

This document is to help you check if CADET is installed correctly.
If you install everything is this document you will be able to run all the examples on your local machine and be ready to use CADET for running further simulations. 

If you need help with the install process please contact us.

## Update the tutorials

This cell will connect to github and update to a newer version of the tutorial later.

**This will update to the latest version which will also remove any local changes you may have made to the tutorial files.**

In [ ]:
# Python path library support
from pathlib import Path
from git import Repo

# This is where the tutorial will be checked out
tutorial_dir = Path.home() / "CADET-Tutorial"

# If the tutorial has already been copied we need to update it instead of copying it again
if tutorial_dir.exists():
    repo = Repo(tutorial_dir.as_posix())

    remote = repo.remote()
    remote.fetch()    
    repo.git.reset('--hard','origin/master')
else:
    Repo.clone_from("https://github.com/modsim/CADET-Tutorial", tutorial_dir)

## CADET-Python
In this tutorial, we will use CADET-Python to interface CADET. It is a file based frontend for CADET which maps almost exactly to the documented CADET [interface specifications](https://cadet.github.io/interface/). The package includes a Cadet class which serves as a generic HDF5 frontend.

As an example, we consider setting the external porosity for the column model (unit_001). From file format, the path for this is /input/model/unit_001/COL_POROSITY. In the Python frontend, this becomes:

```
sim = Cadet()
sim.root.input.model.unit_001.col_porosity = 0.33
```

<div class="alert alert-danger">

**Warning:** 

Note, that the Cadet class does not provide any sanity checks. If parameters are misspelled or have the wrong dimensions, they are simply ignored. This can cause problems later on, when the simulator is run.

</div>


## Utility functions for this workshop

For the purpose of this workshop, we have provided some [utility functions](./utils.ipynb). 

<div class="alert alert-info">

**Note:** 

Please make sure to enter the correct path to your CADET installation there!

</div>

## Verify CADET is working
CADET comes with some basic tests to verify that it is working correctly. 
If you install from the zip file this should just automatically work.
If it does not please report any error messages you get so we can help fix the problem.

In [ ]:
%run ./utils.ipynb

if platform.system() == 'Windows':
    cadet_path = cadet_bin_path / "cadet-cli.exe"
    lwe_path = cadet_bin_path / "createLWE.exe"
else:
    cadet_path = cadet_bin_path / "cadet-cli"
    lwe_path = cadet_bin_path / "createLWE"

if cadet_path.exists() and lwe_path.exists():
    print("CADET was found. Continue to the next step.")
    Cadet.cadet_path = cadet_path.as_posix()
elif cadet_path.exists() and not lwe_path.exists():
    print("CADET was found but createLWE.exe was not found. Please make sure that none of the files have been moved.")
else:
    print("CADET could not be found. Please check the bin path")

ret = subprocess.run([lwe_path.as_posix()], 
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=cadet_bin_path.as_posix())
if ret.returncode == 0:
    print("Test simulation was created")
else:
    print("Failure: Creation of test simulation ran into problems")
if ret.stdout:
    print('Output', ret.stdout.decode('utf-8'))
if ret.stderr:
    print('Errors', ret.stderr.decode('utf-8'))
    
lwe_hdf5_path = cadet_bin_path / 'LWE.h5'

#create a simulation
sim = Cadet()

#set the path to where a simulation can be found
sim.filename = lwe_hdf5_path.as_posix()

#run the simulation
data = sim.run()

if data.returncode == 0:
    print("Simulation completed successfully")
else:
    print("Simulation failed")

#Load the data from the simulation
sim.load()